In [13]:
import pandas as pd
import numpy as np
import os
import utilities
import taylors_pipes as tpipe
import preprocessing as prepro


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


# Our goal here is to use Lasso or Elastic Polynomial Regression to identify which features are relevant

# Read in Data 
#download_path = "/Users/tayma/datasets" # Windows 
download_path = "/Users/taylormurray/datasets" #MaciOS
file_path = os.path.join(download_path, 'housing.csv') #joins the download_path and 'housing.csv' and returns it as a new path
db = pd.read_csv(file_path)

# Create a copy of data
data = db.copy()

# We select our features here (not in pipeline since we are ignoring columns) 

selected_features =['id',
                    'price',
                    'sqfeet', 
                    'beds', 'baths',
                    'cats_allowed',
                    'dogs_allowed',
                    'smoking_allowed'
                    ]

data = data[selected_features]

# Removal of bad values
# Columns with the bad data (might be able to get rid of this)
bad_data_columns = ['price', 
                    'sqfeet', 
                    'baths']

# Dictionary containing the columns with bad data as keys whose items are
# thresholds where values less than the threshold are considered bad data in their respective column.
bad_data_to_replace = { 'price' : 200,
                        'sqfeet' : 50,
                        'baths' : 0 } 

for col,thresh in bad_data_to_replace.items():
    col_mask = data[col] < thresh
    data = data[~col_mask]
    
# IQR Range
# We filter out rows outside the IQR range for each numeric column

data = prepro.IQRFlag(data, disclude = ['id'],invert = True,filter = True )

data




,id,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed
0,7049044568,1148,1078,3,2.0,1,1,0
1,7049047186,1200,1001,2,2.0,0,0,0
2,7043634882,1813,1683,2,2.0,1,1,1
3,7049045324,1095,708,1,1.0,1,1,1
4,7049043759,289,250,0,1.0,1,1,1
...,...,...,...,...,...,...,...,...
384972,7049053337,1295,957,2,2.0,1,1,1
384973,7049052968,1549,1034,2,2.0,1,1,0
384974,7049050454,1249,840,2,1.0,1,1,1
384975,7049050149,1429,976,2,2.0,1,1,1


In [17]:
# Split data
test_set,train_set = utilities.split(data,'id')
X= train_set.drop('price', axis =1)
y=train_set['price']
#Scale with StandardScalar

scale = StandardScaler()
scale.fit(X,y)
X = scale.transform(X)

poly = PolynomialFeatures(degree = 5,include_bias=False)
poly.fit(X,y)
X = poly.transform(X) 
names = poly.get_feature_names_out()

model = Lasso(alpha = 1)
fitted=model.fit(X,y)
pred = fitted.predict(X)
root_mean_squared_error(pred, y)/y.mean()

# Poly



/opt/anaconda3/envs/anaconda-2025.04-py3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.915e+09, tolerance: 3.991e+06
  model = cd_fast.enet_coordinate_descent(


np.float64(0.32021921770585127)

In [46]:
coeff_series = pd.Series(model.coef_)
coeff_series.index = names
zero_coeff_series =coeff_series[coeff_series == 0]
nonzero_coeff_series = coeff_series[(coeff_series < -.25) | (coeff_series >.25)]
nonzero_coeff_series

#coef_db = pd.DataFrame(model.coef_, columns = names)

x0            138.203209
x1            -61.267746
x2             33.233927
x0^2           -7.872487
x0 x2          24.203359
                 ...    
x4^5           -5.204359
x4^3 x5 x6      0.287656
x4 x5^4         1.255081
x4 x6^4        -0.562662
x5^5           -6.747132
Length: 140, dtype: float64